In [ ]:
import sys
import numpy as np
import sklearn_crfsuite
import time
from tqdm import tqdm
import pandas as pd
import prepare_train_with_set as prepare
import os
import func
import set_func

In [ ]:
if __name__ == "__main__":
    set_total = 3
    # How many Set
    ALL = True
    if not ALL:
        set_num = 1
    DEBUG = False
    
    model_name = "crfsuite"
    current_path = os.path.join(os.path.expanduser("~"), "jupyter", "web_verification")
    train_data, Set_dict = prepare.train_file_generate(set_total, current_path)
    test_data = prepare.test_file_generate(current_path)
    
    max_num_train, max_label_train = func.load_data_num(train_data, True)
    max_num_test = func.load_data_num(test_data, False)
    max_num = max(max_num_train, max_num_test)
    
    col_set_dict = dict(map(reversed, Set_dict.items()))
    
    feature_train, label_train, out_train = func.CRFSuite_proccess_data(train_data)
    page_num = int(len(feature_train)/max_num)
    
    crf = sklearn_crfsuite.CRF(
        c1=0.1,
        max_iterations=5
    )
    feature_train = feature_train.tolist()
    label_train = label_train.tolist()
    X_train = [func.sent2features(feature_train)]
    y_train = [func.sent2labels(label_train)]
    
    start = time.time()
    crf.fit(X_train, y_train)
    t = time.time()-start
    
    feature_test, label_test, out_test = func.CRFSuite_proccess_data(test_data)
    feature_test = feature_test.tolist()
    label_test = label_test.tolist()
    X_test = [func.sent2features(feature_test)]
    page_test = int(len(feature_test)/max_num)
    
    ts_start = time.time()
    y_pred = crf.predict(X_test)
    ts = time.time()-ts_start
    
    y_pred = np.array(y_pred)
    result = np.reshape(y_pred, [page_test, max_num])
    result = result.astype(np.int64)
    
    col_type = func.get_col_type(current_path)
    
    Set_data = func.predict_output(set_total, current_path, model_name, col_type, result, max_label_train, col_set_dict)
    set_train_data, set_train_count = func.Set_train_file_generate(set_total, current_path, model_name, feature_train, max_num)
    set_test_data, set_test_count = func.Set_test_file_generate(set_total, current_path, model_name, Set_data, feature_test, max_num)
    page_c = len(result)
    
    if set_total > 0:
        max_num_train = set_func.max_num_set(set_total, set_train_count)
        max_num_test = set_func.max_num_set(set_total, set_test_count)
        max_set = []
        for i in range(len(max_num_train)):
            max_set.append(max(max_num_train[i], max_num_test[i]))

        set_feature_train, set_label_train = set_func.to_train_array_set(set_train_data[0], set_train_count, 0, max_set)

# Run ALL

In [ ]:
if set_total > 0:
    for num in range(set_total):
        set_num = num + 1
        # Load Train file & Test file
        df = get_df("./crfsuite/set/Set-"+str(set_num)+"_train_raw.csv")
        max_num = max_set[set_num-1]
        max_label = max(df['Label'])
        feature_train, label_train = to_train_array_set(df, set_train_count, set_num)
        page_num = int(len(feature_train)/max_num)
        crf = sklearn_crfsuite.CRF(
            c1=0.1
        )
        feature_train = feature_train.tolist()
        label_train = label_train.tolist()
        X_train = [sent2features(feature_train)]
        y_train = [sent2labels(label_train)]
        
        # Train
        start = time.time()
        crf.fit(X_train, y_train)
        t += time.time()-start
        
        # Load Test file
        df = get_df("./crfsuite/set/Set-"+str(set_num)+"_ytest_raw.csv")
        feature_test, label_test = to_train_array_set(df, set_test_count, set_num)
        feature_test = feature_test.tolist()
        label_test = label_test.tolist()
        X_test = [sent2features(feature_test)]
        page_test = int(len(feature_test)/max_num)
            
        # Prediction
        labels = list(crf.classes_)
        ts_start = time.time()
        y_pred = crf.predict(X_test)
        ts += time.time()-ts_start
        y_pred = np.array(y_pred)
        result = np.reshape(y_pred, [page_test, max_num])
        result = result.astype(np.int64)
        
        # Read Col
        col_type = []
        with open("./set/Set-"+str(set_num)+"_coltype.txt", "r") as file:
            tmp = file.readline()
            slot = eval(tmp)
            col_type = slot
            
        # Output
        Set = []
        with open("./crfsuite/set/set-"+str(set_num)+".csv", "w") as file: # Create prediction file
            for col in col_type: # loop to write the Col type
                file.write(col + "\t")
                if DEBUG:
                    print(col + "\t", end='')
            if DEBUG:
                print("")
            file.write("\n")
            current_pos = 1
            for page in tqdm(range(len(result))): # Loop each page
                p_tmp = []
                for cols in range(max_label+1):
                    c_tmp = []
                    for node in range(len(result[page])):
                        r = result[page][node]
                        if r == cols:
                            c_tmp.append(node)
                    p_tmp.append(c_tmp)
                Set.append(p_tmp)
            Set_tmp = Set.copy()
            for page in range(len(Set_tmp)):
                empty = False
                col = []
                for i in range(len(Set_tmp[page])):
                    col.append(False)
                col[0] = True
                while(not empty):
                    for cols in range(len(Set_tmp[page])):
                        if len(Set_tmp[page][cols]) == 0:
                            col[cols] = True
                            if cols != 0:
                                if DEBUG:
                                    print("\t", end="")
                                file.write("\t")
                        else:
                            n = str(int(feature_test[page*max_num+Set_tmp[page][cols][0]][0]))
                            if cols != 0:
                                if DEBUG:
                                    print(n+"\t", end="")
                                file.write(n+"\t")
                            del Set_tmp[page][cols][0]
                            if len(Set_tmp[page][cols]) == 0:
                                col[cols] = True
                        empty = True
                        for i in col:
                            if i == False:
                                empty = False
                                break
                    if DEBUG:
                        print("\n", end="")
                    file.write("\n")

In [ ]:
timef = open("./crfsuite/data/time_crf.txt","w")
print("\ntrain time:"+str(t))
timef.write("train:"+str(t)+"\n")
print("test time:"+str(ts))
print("per page:"+ str(float(ts)/page_c)+"\n")
timef.write("test:"+str(ts)+"\n")
timef.write("per page:"+ str(float(ts)/page_c)+"\n")
timef.close()

In [ ]:
#S56 500